In [ ]:
#  Gestionnaire de Tâches - POO + JSON

import json
from datetime import datetime

FICHIER = "taches.json"

# ================================
# CLASSE TASK
# ================================

class Task:

    def __init__(self, id, titre, priorite="moyenne", statut="à faire"):
        self.id       = id
        self.titre    = titre
        self.priorite = priorite    # "haute" / "moyenne" / "basse"
        self.statut   = statut      # "à faire" / "en cours" / "terminée"
        self.date     = datetime.now().strftime("%d/%m/%Y %H:%M")

    def terminer(self):
        self.statut = "terminée"

    def demarrer(self):
        self.statut = "en cours"

    def vers_dict(self):
        """Convertit l'objet en dictionnaire (pour sauvegarde JSON)"""
        return {
            "id"      : self.id,
            "titre"   : self.titre,
            "priorite": self.priorite,
            "statut"  : self.statut,
            "date"    : self.date
        }

    def __str__(self):
        """Affichage lisible de la tâche"""
        icones_priorite = {"haute": "😡", "moyenne": "😊", "basse": "🥹"}
        icones_statut   = {"à faire": "⬜", "en cours": "🔄", "terminée": "✅"}

        p = icones_priorite.get(self.priorite, "😊")
        s = icones_statut.get(self.statut,     "✅")

        return f"  [{self.id:02d}] {s} {self.titre:<30} {p} {self.priorite:<8}   {self.date}"


# ================================
#  CLASSE TASKMANAGER
# ================================

class TaskManager:

    def __init__(self):
        self.taches     = []
        self.prochain_id = 1
        self.charger()              # charge les tâches au démarrage

    # ── CRUD ───────────────────────────────────────────────

    def ajouter(self, titre, priorite="moyenne"):
        """Crée et ajoute une nouvelle tâche"""
        tache = Task(self.prochain_id, titre, priorite)
        self.taches.append(tache)
        self.prochain_id += 1
        self.sauvegarder()
        print(f" Tâche ajoutée : '{titre}'")

    def supprimer(self, id):
        """Supprime une tâche par son ID"""
        for tache in self.taches:
            if tache.id == id:
                self.taches.remove(tache)
                self.sauvegarder()
                print(f"  Tâche [{id:02d}] supprimée.")
                return
        print(f" Tâche [{id:02d}] introuvable !")

    def changer_statut(self, id, nouveau_statut):
        """Change le statut d'une tâche"""
        statuts_valides = ["à faire", "en cours", "terminée"]

        if nouveau_statut not in statuts_valides:
            print(f" Statut invalide ! Choisissez : {statuts_valides}")
            return

        for tache in self.taches:
            if tache.id == id:
                tache.statut = nouveau_statut
                self.sauvegarder()
                print(f" Tâche [{id:02d}] → '{nouveau_statut}'")
                return
        print(f" Tâche [{id:02d}] introuvable !")

    # ── AFFICHAGE ──────────────────────────────────────────

    def afficher_tout(self):
        """Affiche toutes les tâches"""
        if not self.taches:
            print(" Aucune tâche enregistrée.")
            return

        print(f"\n  {'ID':<6} {'STATUT':<4} {'TITRE':<32} {'PRIORITÉ'}")
        print("  " + "-" * 58)
        for tache in self.taches:
            print(tache)                           # appelle __str__ automatiquement

    def afficher_par_statut(self, statut):
        """Filtre et affiche les tâches par statut"""
        filtrees = [t for t in self.taches if t.statut == statut]

        print(f"\n  Tâches '{statut}' ({len(filtrees)}) :")
        print("  " + "-" * 55)

        if not filtrees:
            print("  Aucune tâche.")
        for tache in filtrees:
            print(tache)

    def afficher_stats(self):
        """Affiche un résumé statistique"""
        total      = len(self.taches)
        a_faire    = sum(1 for t in self.taches if t.statut == "à faire")
        en_cours   = sum(1 for t in self.taches if t.statut == "en cours")
        terminees  = sum(1 for t in self.taches if t.statut == "terminée")
        hautes     = sum(1 for t in self.taches if t.priorite == "haute")

        print(f"\n STATISTIQUES")
        print(f"  {'─' * 30}")
        print(f"  Total tâches    : {total}")
        print(f" À faire      : {a_faire}")
        print(f" En cours     : {en_cours}")
        print(f" Terminées    : {terminees}")
        print(f" Priorité haute : {hautes}")

        if total > 0:
            progression = (terminees / total) * 100
            barre = "" * int(progression // 10) + "░" * (10 - int(progression // 10))
            print(f"\n  Progression : [{barre}] {progression:.0f}%")

    # ── SAUVEGARDE JSON ────────────────────────────────────

    def sauvegarder(self):
        """Sauvegarde toutes les tâches dans un fichier JSON"""
        donnees = {
            "prochain_id": self.prochain_id,
            "taches"     : [t.vers_dict() for t in self.taches]   # list comprehension
        }
        with open(FICHIER, "w", encoding="utf-8") as f:
            json.dump(donnees, f, ensure_ascii=False, indent=2)    # indent=2 → joli formatage

    def charger(self):
        """Charge les tâches depuis le fichier JSON au démarrage"""
        try:
            with open(FICHIER, "r", encoding="utf-8") as f:
                donnees = json.load(f)

            self.prochain_id = donnees["prochain_id"]

            for d in donnees["taches"]:
                tache        = Task(d["id"], d["titre"], d["priorite"], d["statut"])
                tache.date   = d["date"]
                self.taches.append(tache)

            print(f" {len(self.taches)} tâche(s) chargée(s) depuis '{FICHIER}'")

        except FileNotFoundError:
            print(" Nouveau fichier de tâches créé.")


# ================================
#  MENU PRINCIPAL
# ================================

def afficher_menu():
    print("\n" + "=" * 45)
    print(" GESTIONNAIRE DE TÂCHES")
    print("=" * 45)
    print("  1 → Ajouter une tâche")
    print("  2 → Afficher toutes les tâches")
    print("  3 → Changer le statut d'une tâche")
    print("  4 → Filtrer par statut")
    print("  5 → Supprimer une tâche")
    print("  6 → Statistiques")
    print("  7 → Quitter")
    print("=" * 45)


def main():
    manager = TaskManager()

    while True:
        afficher_menu()
        choix = input("Votre choix : ").strip()

        if choix == "1":
            titre    = input("  Titre : ").strip()
            print("  Priorité : 1=haute  2=moyenne  3=basse")
            p        = input("  Priorité : ").strip()
            priorite = {"1": "haute", "2": "moyenne", "3": "basse"}.get(p, "moyenne")
            manager.ajouter(titre, priorite)

        elif choix == "2":
            manager.afficher_tout()

        elif choix == "3":
            try:
                id = int(input("  ID de la tâche : "))
                print("  1=à faire  2=en cours  3=terminée")
                s  = input("  Nouveau statut : ").strip()
                statut = {"1": "à faire", "2": "en cours", "3": "terminée"}.get(s)
                if statut:
                    manager.changer_statut(id, statut)
            except ValueError:
                print(" ID invalide !")

        elif choix == "4":
            print("  1=à faire  2=en cours  3=terminée")
            s      = input("  Statut à filtrer : ").strip()
            statut = {"1": "à faire", "2": "en cours", "3": "terminée"}.get(s)
            if statut:
                manager.afficher_par_statut(statut)

        elif choix == "5":
            try:
                id = int(input("  ID à supprimer : "))
                manager.supprimer(id)
            except ValueError:
                print(" ID invalide !")

        elif choix == "6":
            manager.afficher_stats()

        elif choix == "7":
            print("\n À bientôt !")
            break

        else:
            print(" Choix invalide !")

main()


 1 tâche(s) chargée(s) depuis 'taches.json'

 GESTIONNAIRE DE TÂCHES
  1 → Ajouter une tâche
  2 → Afficher toutes les tâches
  3 → Changer le statut d'une tâche
  4 → Filtrer par statut
  5 → Supprimer une tâche
  6 → Statistiques
  7 → Quitter


Votre choix :  1
  Titre :  Jouer au foot


  Priorité : 1=haute  2=moyenne  3=basse


  Priorité :  basse


 Tâche ajoutée : 'Jouer au foot'

 GESTIONNAIRE DE TÂCHES
  1 → Ajouter une tâche
  2 → Afficher toutes les tâches
  3 → Changer le statut d'une tâche
  4 → Filtrer par statut
  5 → Supprimer une tâche
  6 → Statistiques
  7 → Quitter


...

  Priorité : 1=haute  2=moyenne  3=basse


  Priorité :  Moyenne


 Tâche ajoutée : 'Aller au marché'

 GESTIONNAIRE DE TÂCHES
  1 → Ajouter une tâche
  2 → Afficher toutes les tâches
  3 → Changer le statut d'une tâche
  4 → Filtrer par statut
  5 → Supprimer une tâche
  6 → Statistiques
  7 → Quitter


Votre choix :  2



  ID     STATUT TITRE                            PRIORITÉ
  ----------------------------------------------------------
  [01] ⬜ Apprendre les cours            😊 moyenne    20/02/2026 15:48
  [02] ⬜ Jouer au foot                  😊 moyenne    20/02/2026 15:52
  [03] ⬜ Aller au marché                😊 moyenne    20/02/2026 15:52

 GESTIONNAIRE DE TÂCHES
  1 → Ajouter une tâche
  2 → Afficher toutes les tâches
  3 → Changer le statut d'une tâche
  4 → Filtrer par statut
  5 → Supprimer une tâche
  6 → Statistiques
  7 → Quitter
